# 🐼 07. pandas 심화 — 데이터 분석 & Excel 입출력

> **목표**: pandas로 대용량 데이터를 분석하고, 피벗/그룹바이/필터링 결과를 xlwings와 연동해서 Excel로 깔끔하게 출력한다.
>
> **왜 중요한가?** — xlwings만으로는 데이터 분석이 어렵습니다. **pandas + xlwings 조합**이 실무 최강입니다.

---

### 📚 이 챕터에서 배우는 것

| 순서 | 주제 | 실무 적용 |
|:---:|:---|:---|
| 1 | DataFrame 기초 | 표 형태로 데이터 다루기 |
| 2 | 조건 필터링 | 기준에 맞는 행만 추출 |
| 3 | groupby 집계 | 부서/팀별 합계/평균 |
| 4 | 피벗 테이블 | 교차 집계표 자동 생성 |
| 5 | Excel 읽기/쓰기 | `read_excel` & `to_excel` |
| 6 | xlwings 연동 | 분석 결과를 서식 있게 출력 |
| 7 | 🔥 실전 종합 | 전체 파이프라인 자동화 |

---
## 0️⃣ 준비: 설치 & 임포트

```bash
pip install pandas openpyxl xlwings
```

In [ ]:
import pandas as pd
import xlwings as xw
import numpy as np
from pathlib import Path

print(f'✅ pandas  버전: {pd.__version__}')
print(f'✅ xlwings 버전: {xw.__version__}')
print(f'✅ numpy   버전: {np.__version__}')

In [ ]:
# 실습용 샘플 DataFrame 생성
import random

random.seed(42)
departments = ['영업팀', '마케팅팀', '개발팀', '인사팀']
names = ['김영희', '이철수', '박민수', '정지영', '최수진',
         '홍길동', '강다솔', '임서연', '윤준혁', '신예린',
         '오태양', '배소연', '문현우', '류아린', '한지운']

data = {
    '이름': names,
    '부서': [random.choice(departments) for _ in names],
    '입사연도': [random.randint(2018, 2024) for _ in names],
    '교육시간': [random.randint(4, 25) for _ in names],
    '성과점수': [random.randint(60, 100) for _ in names],
    '급여': [random.randint(3000, 7000) * 1000 for _ in names],
}

df = pd.DataFrame(data)

print(f'📊 샘플 데이터: {len(df)}행 × {len(df.columns)}열')
print(f'📋 컬럼: {list(df.columns)}')
display(df.head())

---
## 1️⃣ DataFrame 기초 조작

> pandas의 핵심 자료구조 `DataFrame` — 엑셀의 표와 동일한 개념입니다.

In [ ]:
# 기본 정보 확인
print('=== 기본 정보 ===')
print(df.info())

print('\n=== 기술 통계 ===')
display(df.describe())

In [ ]:
# 열 선택
print('▶ 단일 열:')
print(df['부서'].values)

print('\n▶ 여러 열:')
display(df[['이름', '부서', '교육시간']].head())

# 행 선택 (인덱스 기반)
print('\n▶ 0~2행:')
display(df.iloc[0:3])

In [ ]:
# 정렬
df_sorted = df.sort_values('성과점수', ascending=False)
print('▶ 성과점수 내림차순:')
display(df_sorted.head())

# 여러 컬럼으로 정렬
df_sorted2 = df.sort_values(['부서', '교육시간'], ascending=[True, False])
print('\n▶ 부서 오름차순 → 교육시간 내림차순:')
display(df_sorted2.head())

---
## 2️⃣ 조건 필터링

> "교육시간 15시간 이상인 직원만 추출" — 실무에서 가장 자주 쓰는 기능입니다.

In [ ]:
# 단순 조건
high_edu = df[df['교육시간'] >= 15]
print(f'▶ 교육시간 15시간 이상: {len(high_edu)}명')
display(high_edu)

In [ ]:
# 복합 조건
# & = AND,  | = OR,  ~ = NOT

# 영업팀이면서 성과점수 80 이상
result1 = df[(df['부서'] == '영업팀') & (df['성과점수'] >= 80)]
print(f'▶ 영업팀 & 성과점수 80+: {len(result1)}명')
display(result1)

# 영업팀 또는 마케팅팀
result2 = df[df['부서'].isin(['영업팀', '마케팅팀'])]
print(f'\n▶ 영업팀 or 마케팅팀: {len(result2)}명')

In [ ]:
# query() 방식 → 읽기 쉬운 문자열 조건
result3 = df.query('교육시간 >= 15 and 성과점수 >= 80')
print(f'▶ query() 방식 — 교육시간 15+ & 성과점수 80+: {len(result3)}명')
display(result3)

# 새 컬럼 추가 (파생 변수)
df['등급'] = df['성과점수'].apply(
    lambda x: 'A' if x >= 90 else ('B' if x >= 80 else 'C')
)
print('\n▶ 등급 컬럼 추가:')
display(df[['이름', '성과점수', '등급']].head())

---
## 3️⃣ groupby — 그룹별 집계

> Excel의 "부서별 소계" 기능을 코드 한 줄로!  
> `df.groupby('부서')['열'].agg(함수)`

In [ ]:
# 부서별 교육시간 합계
group1 = df.groupby('부서')['교육시간'].sum()
print('▶ 부서별 교육시간 합계:')
print(group1)

# 부서별 인원 수
group2 = df.groupby('부서')['이름'].count()
print('\n▶ 부서별 인원수:')
print(group2)

In [ ]:
# 여러 집계 함수 동시 적용
summary = df.groupby('부서').agg(
    인원수=('이름', 'count'),
    교육시간_합계=('교육시간', 'sum'),
    교육시간_평균=('교육시간', 'mean'),
    성과점수_평균=('성과점수', 'mean'),
    급여_평균=('급여', 'mean')
).round(1).reset_index()

print('▶ 부서별 종합 통계:')
display(summary)

In [ ]:
# 다중 groupby (부서 + 등급)
group3 = df.groupby(['부서', '등급']).size().reset_index(name='인원수')
print('▶ 부서 × 등급별 인원수:')
display(group3)

---
## 4️⃣ 피벗 테이블

> Excel의 피벗 테이블을 코드로!  
> `pd.pivot_table(df, values, index, columns, aggfunc)`

In [ ]:
# 부서별 × 등급별 평균 교육시간 피벗
pivot1 = pd.pivot_table(
    df,
    values='교육시간',
    index='부서',        # 행
    columns='등급',      # 열
    aggfunc='mean',      # 집계 함수
    fill_value=0         # NaN → 0
).round(1)

print('▶ 부서 × 등급별 평균 교육시간:')
display(pivot1)

In [ ]:
# margins=True → 합계 행/열 추가 (소계)
pivot2 = pd.pivot_table(
    df,
    values='성과점수',
    index='부서',
    columns='등급',
    aggfunc='count',
    fill_value=0,
    margins=True,       # 합계 추가
    margins_name='합계'
)

print('▶ 부서×등급 인원 피벗 (합계 포함):')
display(pivot2)

---
## 5️⃣ Excel 파일 읽기/쓰기

| 기능 | pandas 함수 | 설명 |
|:---|:---|:---|
| Excel → DataFrame | `pd.read_excel()` | xlsx 파일을 바로 DataFrame으로 |
| DataFrame → Excel | `df.to_excel()` | DataFrame을 xlsx로 저장 |
| 여러 시트 읽기 | `sheet_name=None` | 딕셔너리로 전체 시트 반환 |

In [ ]:
# DataFrame → Excel 저장
OUTPUT_PATH = 'pandas_output.xlsx'

with pd.ExcelWriter(OUTPUT_PATH, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='원본데이터', index=False)
    summary.to_excel(writer, sheet_name='부서별요약', index=False)
    pivot1.to_excel(writer, sheet_name='피벗테이블')

print(f'✅ 저장 완료: {OUTPUT_PATH}')
print(f'   시트: 원본데이터 / 부서별요약 / 피벗테이블')

In [ ]:
# Excel → DataFrame 읽기
df_read = pd.read_excel(OUTPUT_PATH, sheet_name='원본데이터')
print(f'✅ 읽기 완료: {len(df_read)}행 × {len(df_read.columns)}열')
display(df_read.head())

# 여러 시트 한번에 읽기
all_sheets = pd.read_excel(OUTPUT_PATH, sheet_name=None)
print(f'\n📋 읽은 시트 목록: {list(all_sheets.keys())}')

In [ ]:
# 읽기 옵션들
df_opts = pd.read_excel(
    OUTPUT_PATH,
    sheet_name='원본데이터',
    header=0,           # 0번 행을 헤더로
    usecols='A:D',      # A~D열만 읽기
    nrows=5,            # 5행만 읽기
    dtype={'성과점수': float}  # 데이터 타입 지정
)

print('▶ usecols=A:D, nrows=5 로 읽기:')
display(df_opts)

---
## 6️⃣ pandas ↔ xlwings 연동 (핵심!)

> **pandas로 분석** + **xlwings로 서식 입혀서 출력** = 실무 최강 콤보

| 방향 | 코드 | 설명 |
|:---|:---|:---|
| Excel → DataFrame | `.options(pd.DataFrame, header=1).value` | xlwings에서 pandas로 |
| DataFrame → Excel | `.options(pd.DataFrame).value = df` | pandas 결과를 엑셀에 출력 |

In [ ]:
import xlwings as xw
import pandas as pd

app = xw.App(add_book=False)
wb = app.books.add()

# ─── Excel → DataFrame ───
ws_in = wb.sheets[0]
ws_in.name = '원본'

# 데이터 직접 기입
ws_in.range('A1').value = [['이름', '부서', '교육시간', '성과점수']] + \
    [[row['이름'], row['부서'], row['교육시간'], row['성과점수']] for _, row in df.iterrows()]

# xlwings → pandas DataFrame
df_from_xw = ws_in.range('A1').expand('table').options(
    pd.DataFrame, header=1
).value

print(f'✅ xlwings → pandas: {len(df_from_xw)}행')
display(df_from_xw.head())

In [ ]:
# ─── DataFrame → Excel (xlwings 방식) ───
ws_out = wb.sheets.add('분석결과')

# 제목
ws_out.range('A1').value = '📊 부서별 종합 분석 결과'
ws_out.range('A1').font.bold = True
ws_out.range('A1').font.size = 14
ws_out.range('A1:F1').merge()
ws_out.range('A1').color = (47, 85, 151)
ws_out.range('A1').font.color = (255, 255, 255)
ws_out.range('A1').row_height = 40
ws_out.range('A1').api.HorizontalAlignment = -4108

# summary DataFrame 출력 (헤더 포함)
ws_out.range('A3').options(pd.DataFrame, index=False).value = summary

# 헤더 서식
header_range = ws_out.range('A3').expand('right')
header_range.font.bold = True
header_range.color = (68, 114, 196)
header_range.font.color = (255, 255, 255)
header_range.api.HorizontalAlignment = -4108

# 열 너비 자동
ws_out.range('A3').expand('table').columns.autofit()

print('✅ DataFrame → Excel 출력 완료')
print('💡 엑셀에서 분석결과 시트를 확인해보세요!')

---
## 🔥 7. 실전 종합: 전체 파이프라인 자동화

**시나리오**: Excel 원본 파일 → pandas 분석 → 요약/피벗/필터 결과를 별도 시트로 출력

```
흐름
────
원본.xlsx 열기
    │
    ▼
xlwings → pandas DataFrame 변환
    │
    ├─→ [분석①] groupby 집계 → '부서요약' 시트
    ├─→ [분석②] 피벗 테이블 → '피벗' 시트  
    └─→ [분석③] 조건 필터 (우수직원) → '우수직원' 시트
    │
    ▼
서식 적용 후 저장 → 완료
```

In [ ]:
def apply_table_style(ws, start_cell='A1', title=None):
    """DataFrame 출력 후 표준 서식 적용 헬퍼 함수"""
    rng = ws.range(start_cell).expand('table')
    
    # 헤더 서식
    header = ws.range(start_cell).expand('right')
    header.font.bold = True
    header.color = (68, 114, 196)
    header.font.color = (255, 255, 255)
    header.api.HorizontalAlignment = -4108
    header.row_height = 28
    
    # 데이터 줄무늬
    last_row = rng.last_cell.row
    last_col = ws.range(start_cell).expand('right').last_cell.address.split('$')[1]
    start_row = ws.range(start_cell).row
    for row in range(start_row + 1, last_row + 1):
        if row % 2 == 0:
            ws.range(f'A{row}:{last_col}{row}').color = (221, 235, 247)
    
    # 테두리
    for border_id in range(7, 13):
        try:
            rng.api.Borders(border_id).LineStyle = 1
        except:
            pass
    
    # 열 너비
    rng.columns.autofit()
    
    # 시트 제목
    if title:
        ws.range('A1').api.entireRow.Insert()
        ws.range('A1').value = title
        ws.range('A1').font.bold = True
        ws.range('A1').font.size = 13
        ws.range('A1').row_height = 36


print('✅ apply_table_style() 함수 정의 완료')

In [ ]:
def full_analysis_pipeline(df_input):
    """pandas 분석 → xlwings로 Excel 출력 전체 파이프라인"""
    
    app_p = xw.App(add_book=False)
    wb_p = app_p.books.add()
    
    # 기본 시트 제거 후 시작
    default_sheet = wb_p.sheets[0]
    
    try:
        # ─── 시트 1: 원본 데이터 ───────────────────────
        default_sheet.name = '원본데이터'
        ws1 = default_sheet
        ws1.range('A1').options(pd.DataFrame, index=False).value = df_input
        apply_table_style(ws1)
        print('✅ [1/4] 원본 데이터 시트 완료')
        
        # ─── 시트 2: 부서별 요약 ──────────────────────
        ws2 = wb_p.sheets.add('부서요약', after=ws1)
        dept_summary = df_input.groupby('부서').agg(
            인원수=('이름', 'count'),
            평균교육시간=('교육시간', 'mean'),
            평균성과점수=('성과점수', 'mean'),
            평균급여=('급여', 'mean')
        ).round(1).reset_index()
        
        ws2.range('A1').options(pd.DataFrame, index=False).value = dept_summary
        apply_table_style(ws2)
        print('✅ [2/4] 부서요약 시트 완료')
        
        # ─── 시트 3: 피벗 테이블 ────────────────────
        ws3 = wb_p.sheets.add('피벗', after=ws2)
        pivot = pd.pivot_table(
            df_input, values='성과점수',
            index='부서', columns='등급',
            aggfunc='count', fill_value=0,
            margins=True, margins_name='합계'
        ).reset_index()
        
        ws3.range('A1').options(pd.DataFrame, index=False).value = pivot
        apply_table_style(ws3)
        print('✅ [3/4] 피벗 시트 완료')
        
        # ─── 시트 4: 우수 직원 ──────────────────────
        ws4 = wb_p.sheets.add('우수직원', after=ws3)
        best = df_input.query('성과점수 >= 85 and 교육시간 >= 12').sort_values(
            '성과점수', ascending=False
        )
        ws4.range('A1').options(pd.DataFrame, index=False).value = best
        apply_table_style(ws4)
        print(f'✅ [4/4] 우수직원 시트 완료 ({len(best)}명)')
        
        # ─── 저장 ────────────────────────────────────
        save_path = 'analysis_result.xlsx'
        wb_p.save(save_path)
        print(f'\n💾 저장: {save_path}')
        print('🎉 전체 파이프라인 완료!')
        
    except Exception as e:
        print(f'❌ 에러: {e}')
        raise


# 실행!
full_analysis_pipeline(df)

---
## 💾 마무리

In [ ]:
# 열린 엑셀 앱 종료 (확인 후 실행)
# app.quit()

print('💡 엑셀 확인 후 app.quit() 실행하세요')

---
## ⚠️ 흔한 에러와 해결

| 증상 | 원인 | 해결 |
|:---|:---|:---|
| `ModuleNotFoundError: openpyxl` | openpyxl 미설치 | `pip install openpyxl` |
| `read_excel` 결과 NaN 많음 | 빈 셀 또는 헤더 행 불일치 | `header=0` 또는 `header=None` 조정 |
| `groupby` 후 인덱스가 이상함 | `reset_index()` 누락 | `.reset_index()` 붙이기 |
| 피벗 값이 NaN | 교차되는 데이터 없음 | `fill_value=0` 지정 |
| `options(pd.DataFrame)` 오류 | pandas 미임포트 | `import pandas as pd` 확인 |
| 한글 열 이름 깨짐 | 인코딩 문제 | `encoding='utf-8-sig'` 지정 (CSV의 경우) |

---
## 📝 연습 문제

1. `df`에서 **"입사연도가 2021년 이전"이面서 "급여 5,000,000 이상"**인 직원만 필터링하세요
2. 부서별로 **교육시간 합계 / 급여 합계**를 구하는 `groupby`를 작성하세요
3. `full_analysis_pipeline()`에 "저조직원" 시트(성과 70점 미만)를 추가해서 빨간색 서식을 적용해 보세요

In [ ]:
# 🏋️ 연습 1: 조건 필터링
# 입사연도 2021년 이전 AND 급여 5000000 이상

# 여기에 코드 작성!


In [ ]:
# 🏋️ 연습 2: groupby
# 부서별 교육시간 합계 / 급여 합계

# 여기에 코드 작성!


---
## 📌 다음 챕터 예고

> **08. openpyxl — 서식 완전 제어**  
> 조건부 서식, 이미지 삽입, 드롭다운 목록, 데이터 유효성 검사 등  
> xlwings가 못하는 세밀한 서식을 openpyxl로 완성합니다.